<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/simulation_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
rm(list = ls())

##***Load Dataset***

In [20]:
#load dataset
simulation_data = read.table("sim_data.csv", header= TRUE, sep=",")
N = ncol(simulation_data) #N = number columns (samples in the dataset)
n_gene = nrow(simulation_data) #n_gene = no of genes (rows in the dataset)

In [21]:
head(simulation_data)

,GSM1247203,GSM1247204,GSM1247205,GSM1247206,GSM1247207,GSM1247208,GSM1247209,GSM1247210,GSM1247211,GSM1247212,⋯,GSM1247410,GSM1247411,GSM1247412,GSM1247413,GSM1247414,GSM1247415,GSM1247416,GSM1247417,GSM1247418,GSM1247419
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4.319644,4.396137,3.968531,3.979268,3.901083,4.207761,3.963131,4.164432,3.873857,4.217890,⋯,4.484714,4.219507,3.925787,3.861954,4.015559,4.623454,5.223074,3.927950,3.798007,3.958021
2,9.608635,9.244517,10.553440,9.754355,8.997556,9.027160,9.227125,9.198466,8.908276,9.604081,⋯,8.846296,8.666471,9.462380,9.199890,9.022814,9.072050,9.679663,8.615543,8.796901,8.688204
3,5.923612,6.797588,6.369745,5.671180,6.077140,6.341990,6.686084,6.154192,5.906662,6.186725,⋯,6.820596,6.006575,6.250236,6.047955,6.467733,6.707928,7.916406,5.589499,5.016101,5.278193
4,9.911118,9.421990,9.358638,9.275714,8.573832,9.072234,9.053680,9.549565,9.458813,9.247518,⋯,10.028440,9.816180,8.794598,9.688042,8.914406,9.211488,9.399957,9.693126,8.551371,9.780800
5,6.292236,6.047995,6.268707,6.275980,6.257831,6.371053,6.149960,6.261317,6.361110,6.336733,⋯,6.278971,6.549239,6.395798,6.263772,6.119008,6.146906,6.276937,6.206087,6.438444,6.251627
6,7.310085,8.409503,7.802326,7.260794,7.598630,7.504309,8.223495,7.424841,7.065065,7.577485,⋯,7.754192,7.051993,7.425191,7.989666,7.573813,7.743429,8.081255,7.023867,7.322741,7.321942


In [22]:
dim(simulation_data)

[1] 20219   217

In [23]:
set.seed(1)
m = 10000 #no of genes to be selected from the dataset
randomly_selected_genes = sample(n_gene, m) #randomly selected genes of size m from the total genes

head(randomly_selected_genes)
length(randomly_selected_genes)

[1] 17401  4775 13218 10539  8462  4050

[1] 10000

10k randomly selected data for simulation process

In [24]:
#10000 selected data corresponding to the randomly selected genes (EE genes)
randomly_selected_data = simulation_data[randomly_selected_genes, ]
genej_sim_data = randomly_selected_data
head(genej_sim_data)
dim(genej_sim_data)

,GSM1247203,GSM1247204,GSM1247205,GSM1247206,GSM1247207,GSM1247208,GSM1247209,GSM1247210,GSM1247211,GSM1247212,⋯,GSM1247410,GSM1247411,GSM1247412,GSM1247413,GSM1247414,GSM1247415,GSM1247416,GSM1247417,GSM1247418,GSM1247419
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
17401,7.252734,7.303979,7.010113,6.760083,7.459250,7.264806,7.441763,7.080874,7.218505,7.343332,⋯,7.379983,7.181466,7.332173,7.204830,7.089101,7.034548,7.151740,7.135917,7.595081,7.322358
4775,4.165667,4.435168,4.385319,4.420685,4.152833,3.816157,4.201120,4.029131,4.034959,4.177509,⋯,4.109681,4.031004,4.180370,4.046903,4.022233,4.061613,4.244768,3.818283,3.911840,4.099252
13218,7.629774,8.434767,7.971606,8.015123,7.517571,7.230787,7.881967,7.835166,8.319057,8.088978,⋯,8.052246,7.810543,7.634784,8.381040,7.980730,7.325929,7.867981,8.242966,7.700593,8.018120
10539,6.915510,7.562835,7.405954,8.888284,6.957932,6.642156,6.770809,6.772811,6.661028,6.976250,⋯,8.309056,6.928286,6.647726,6.942022,6.752002,6.683944,6.985731,7.734789,6.958129,7.541349
8462,5.949411,5.801627,5.792423,5.666441,5.843078,5.670973,5.809662,5.423051,5.958257,5.824004,⋯,5.784465,5.980613,5.989962,5.781039,6.324430,5.347286,6.174486,5.765319,5.868937,6.113311
4050,10.050940,11.042590,9.510201,9.602699,10.909970,11.115290,11.792050,9.915201,11.798120,10.769450,⋯,9.197424,10.616580,10.694180,10.988510,11.430450,10.100610,11.895710,9.907439,11.307340,9.178329


[1] 10000   217

standarad deviation for 10000 genes from all experimental units

In [25]:
genej_std = apply(genej_sim_data, 1, sd)
head(genej_std)
length(genej_std)

17401      4775     13218     10539      8462      4050 
0.2237894 0.2063794 0.2945551 0.5984411 0.3111447 0.8521204

[1] 10000

## Experimental units - Part 1

Here, we create 2 treatment groups with 4 randomly selected samples each with 10,000 randlomly selected genes.

In [26]:
set.seed(1)
n_samples = 4 #no of samples
randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group

treatment_group1 = randomly_selected_samples[1:n_samples]
treatment_group2 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

treatment_group1
treatment_group2

[1]  68 167 129 162

[1]  43  14 210 187

## Experimental units - Part 2

In [27]:
set.seed(5) #simulation number for exp units
n_samples = 4 #no of samples
randomly_selected_samples  = sample(N, 2*n_samples) #randomly select samples of size 2*n_trt_grp 
                                                    #for treatment group

treatment_group15 = randomly_selected_samples[1:n_samples]
treatment_group25 = randomly_selected_samples[(n_samples + 1):(2*n_samples)]

treatment_group15
treatment_group25

[1]  66 107 185 207

[1] 203 121  41  71

Group 1 data

In [28]:
treatment_group1_data = genej_sim_data[treatment_group15]
head(treatment_group1_data)
write.csv(treatment_group1_data, "10k_untreated_group15.csv")

,GSM1247268,GSM1247309,GSM1247387,GSM1247409
,<dbl>,<dbl>,<dbl>,<dbl>
17401,7.340714,7.394424,7.243478,7.101899
4775,4.394040,4.219564,4.489715,4.206203
13218,8.103082,7.992739,8.204715,7.663785
10539,6.606597,7.475056,6.824796,7.184802
8462,5.841540,6.006771,6.521505,5.825291
4050,11.438090,10.713660,10.065300,10.868320


##**Simulation**

* Group 1 consists of EE genes and group 2 consists of 9k EE genes and 1k DE genes (genes with treatment effects)
* treatment_group2_data consists of the 4 randomly selected samples (exp. units)

In [29]:
treatment_group2_data = genej_sim_data[treatment_group25]
dim(treatment_group2_data)
head(treatment_group2_data)

[1] 10000     4

,GSM1247405,GSM1247323,GSM1247243,GSM1247273
,<dbl>,<dbl>,<dbl>,<dbl>
17401,7.651242,7.306319,7.309459,7.154801
4775,3.900088,3.922012,4.181620,3.828347
13218,7.504424,8.022729,7.835551,7.735613
10539,6.808388,6.921469,7.536829,6.694359
8462,5.956972,6.108686,5.570955,5.662761
4050,11.412720,10.295770,11.124620,11.241340


## **Simulation from 3.1**

We calculate the treatment effect by extracting the effect size of each gene j from a random normal distribution using genej_std (standard deviation for the 10k randomly selected genes from all experimental units in the population data)

In [30]:
set.seed(1)
mu = 2*genej_std

treatment_effect_j = rnorm(10000, mu, genej_std)
length(treatment_effect_j)
head(data.frame(treatment_effect_j))

[1] 10000

,treatment_effect_j
,<dbl>
1,0.3073851
2,0.4506590
3,0.3429716
4,2.1515638
5,0.7248141
6,1.0051030


## **DE genes**

DE genes are created by adding treatment effects extracted to each gene in group 2

In [31]:
dim(treatment_group2_data)
de_treatment_group2 = treatment_group2_data + treatment_effect_j
head(de_treatment_group2)

[1] 10000     4

,GSM1247405,GSM1247323,GSM1247243,GSM1247273
,<dbl>,<dbl>,<dbl>,<dbl>
17401,7.958627,7.613704,7.616844,7.462186
4775,4.350747,4.372671,4.632279,4.279006
13218,7.847396,8.365701,8.178523,8.078585
10539,8.959952,9.073033,9.688393,8.845923
8462,6.681786,6.833500,6.295769,6.387575
4050,12.417823,11.300873,12.129723,12.246443


we randomly selected 1000 genes with treatment effect from de_treatment_group2

In [32]:
set.seed(1)
n_de = 1000 
n_treated_genes_grp2 = nrow(de_treatment_group2)

#we extract 1000 genes from DE_treatment_group2 (genes with treatment effects)
de_genes_index = sample(n_treated_genes_grp2, n_de)

#we copied untreated (genes without effects) genes from treatment_group2_data
treated_genes_group2 = treatment_group2_data

#we replaced the 1000 untreated genes (without treatment effects) with the corresponding treated genes(with treatment effects)
treated_genes_group2[de_genes_index, ] = de_treatment_group2[de_genes_index, ]

dim(treated_genes_group2)
head(de_genes_index)


[1] 10000     4

[1] 1017 8004 4775 9725 8462 4050

In [33]:
head(de_genes_index)

[1] 1017 8004 4775 9725 8462 4050

**Summary of Group 2 simulation**

m00 - number of EE genes (consists of 9000 genes)

m11 - number of DE genes (1000 genes)


In [34]:
m00 = treated_genes_group2[-de_genes_index, ]
m11 = treated_genes_group2[de_genes_index, ]
treatment_effect_size_grp2 = treatment_effect_j[de_genes_index]
write.csv(treated_genes_group2, "10k_treated_group25.csv")
dim(m00)
dim(m11)
length(treatment_effect_size_grp2)

[1] 9000    4

[1] 1000    4

[1] 1000

## Not part

In [35]:
# set.seed(1)
# mu = 2
# for (i in 1:length(df_m11_genes_std[1][,1])){

#   #print(df_m11_genes_std[1][i,1])
  
#   genes_treatment_effect <- rnorm(1000, mu*df_m11_genes_std[1][i,1], df_m11_genes_std[1][i,1])
  
# }

# head(genes_treatment_effect)
# gene_j_effect = data.frame(genes_treatment_effect)
# head(gene_j_effect)
# dim(gene_j_effect)

In [36]:
# set.seed(1)
# gene_j_effect_fn <- function(n, mu, std_data){
#   mu = mu
#   for (i in 1:length(std_data[,1])){
    
#     genes_treatment_effect2 <- rnorm(n, mu*std_data[i,1], std_data[i,1])
#     return (genes_treatment_effect2)
#   }
# }

# head(gene_j_effect_fn(1000, 2, df_m11_genes_std[1]))
